<h1>Inhaltsverzeichnis<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Notwendige-Bibliotheken-und-Funktionalitäten" data-toc-modified-id="Notwendige-Bibliotheken-und-Funktionalitäten-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Notwendige Bibliotheken und Funktionalitäten</a></span></li><li><span><a href="#Allgemeine-Vorbemerkungen" data-toc-modified-id="Allgemeine-Vorbemerkungen-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Allgemeine Vorbemerkungen</a></span></li><li><span><a href="#Vorweg:-Die-Caesar-Chiffre" data-toc-modified-id="Vorweg:-Die-Caesar-Chiffre-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Vorweg: Die Caesar-Chiffre</a></span><ul class="toc-item"><li><span><a href="#Wir-verschlüsseln-mit-Caesar!" data-toc-modified-id="Wir-verschlüsseln-mit-Caesar!-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Wir verschlüsseln mit Caesar!</a></span></li><li><span><a href="#Caesar-decodieren" data-toc-modified-id="Caesar-decodieren-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Caesar decodieren</a></span></li></ul></li><li><span><a href="#Wie-funktioniert-die-Vigenère-Verschlüsselung?" data-toc-modified-id="Wie-funktioniert-die-Vigenère-Verschlüsselung?-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Wie funktioniert die Vigenère-Verschlüsselung?</a></span></li></ul></div>

# Vigenère; Erläuterungen mit einer Python-Umgebung

## Notwendige Bibliotheken und Funktionalitäten

Für die folgenden Programmteile werden einige Python-Bibliotheken benötigt, die jedoch ausschließlich technische Werkzeuge bereitstellen, nicht jedoch notwendig sind für die eigentlichen Verschlüsselungsverfahren.

In [1]:
import ipywidgets as widgets

In [2]:
output=widgets.Output()

## Allgemeine Vorbemerkungen

Jeder Text, also sowohl die zu versendende **Nachricht**, der benutzte **Schlüssel** als auch der daraus resultierende **Geheimtext**, besteht ausschließlich aus den Großbuchstaben!

- Also gibt es insbesondere
   - **keine Umlaute**
   - **keine Ziffern**
   - **keine Satzzeichen**
   - **keine Sonderzeichen**
   - **keine Leerzeichen**
   - **keine Zeilenumbrüche**
   
Folglich kann man den zulässige Zeichenvorrat unter dem Namen `alphabet` definieren:

In [3]:
alphabet = "abcdefghijklmnopqrstuvwxyz".upper()

Damit alle Texte sicher behandelt werden können, gibt es ein passendes Werkzeug:
- Alle Kleinbuchstaben werden in Großbuchstaben umgewandelt.
- Alle Zeichen des Textes, die nicht im Alphabet enthalten sind, werden entfernt.

In [4]:
def normiereText(text):
    normierterText = ""
    for zeichen in text.upper():
        if zeichen in alphabet:
            normierterText += zeichen
    return normierterText    

Zum Test:

In [5]:
normiereText("KJhk  ??Kj92sJH.,:;)8/ ")

'KJHKKJSJH'

## Vorweg: Die Caesar-Chiffre

Die **Caesar-Verschlüsselung** verschiebt zyklisch (also modulo 26) jedes Zeichen der Nachricht um einen durch den **Schlüssel** definierten Wert. Das kann durch die folgende Funktion realisiert werden:

In [6]:
def verschiebe(zeichen, shift=13):
    position = (alphabet.index(zeichen) + shift) % 26
    return alphabet[position]

Eine Nachricht wird dann zeichenweise mit Hilfe dieser Funktion verschlüsselt:

In [7]:
def caesar(nachricht, schluessel=13):
    text = normiereText(nachricht)
    geheim = ""
    for zeichen in text:
        neu = verschiebe(zeichen, schluessel)
        geheim += neu
    return geheim

### Wir verschlüsseln mit Caesar!

Zum Testen mal wieder `HalloWelt` mit Verschiebung 3:

In [51]:
caesar("Hallo Welt", 3)

'KDOORZHOW'

Benutzerfreundlicher kann man das mit sog. Widgets gestalten:

In [62]:
def on_value_change(event):
    output.clear_output()
    with output:
        geheim = caesar(text.value, slider.value)
        print("Caesar-codierung: ", geheim)

In [63]:
slider = widgets.IntSlider(value=0, min=0,max=25,
                            description="Verschiebung")

slider.observe(on_value_change, names="value")

text=widgets.Text(description = "Klartext", value = "Hallo Welt")

text.observe(on_value_change, names="value")

In [70]:
output.clear_output()
display(text, slider, output)

Text(value='Hallo Welt', description='Klartext')

IntSlider(value=0, description='Verschiebung', max=25)

Output(outputs=({'output_type': 'stream', 'text': 'Caesar-codierung:  HALLOWELT\n', 'name': 'stdout'},))

### Caesar decodieren

Um eine Caesar-verschlüsselte Nachricht bei bekanntem Schlüssel `s` zu entschlüsseln, kann man die Nachricht mit dem Schlüssel `26-s` verschlüsseln:

In [65]:
def reCaesar(geheim, schluessel=13):
    text = normiereText(geheim)
    return caesar(text, 26-schluessel)

Jetzt testen wir! 
- Dazu nehmen wir einen Klartext, 
    - verschlüsseln ihn zu einem Geheimtext, 
    - entschlüsseln diesen Geheimtext, 
    - jeweils mit demselben Schlüssel.

Dann müsste der Klartext zurückgewonnen werden!

In [68]:
klartext = "HALLOWELT"
schlüssel = 12
print("Klartext", klartext)
geheimtext = caesar(klartext, schlüssel)
print("Geheimtext", geheimtext)
decodiert = reCaesar(geheimtext, schlüssel)
print("Decodiert", klartext)

Klartext HALLOWELT
Geheimtext TMXXAIQXF
Decodiert HALLOWELT


## Wie funktioniert die Vigenère-Verschlüsselung?

Wenn man einen Text mit dem Vigenère-Verfahren verschlüsselt, wird jeder Buchstabe des Textes mit dem entsprechenden Buchstaben des Schlüssels verarbeitet.

Benutzt man die Nummer des Schlüsselbuchstabens im Alphabet (also A=0, B=1, ...) statt des Buchstabens selber, so entpuppt sich die Vigenère-Verschlüsselung eines Zeichens als Caesar-Verschiebung.

Will man z.B. den Text `HALLOWELT` mit dem Schlüssel `SCH` per Vigenère verschlüsseln, so wird z.B. der Buchstabe `H` mit `S` verarbeitet. Also wird das `H` um 18 Stellen (S=18) verschoben. Das `H` wird dann also mit dem Buchstaben `Z` verschlüsselt:

`H = 8; S = 18; 8+18 = 24; 24 = Z`</p>

    

    


Damit ist also die Vigenère-Verschlüsselung eine Art Weiterentwicklung der Caesar-Verschlüsselung.

Also wird man zunächst die zu einem (Groß-)buchstaben gehörige Nummer im Alphabet finden müssen:

In [12]:
def indexVonZeichen(zeichen):
    return alphabet.index (zeichen)

In [13]:
indexVonZeichen("S")

18

Wenn man die Verschiebung mit Hilfe eines Zeichens (statt der Verschiebiungsdistanz) nutzen möchte, so wie später für die Vigenère-Chiffre notwendig, nutzt man die folgende Definition:

In [14]:
def vigVerschiebe(zeichen, schluesselzeichen):
    shift = indexVonZeichen(schluesselzeichen)  
    return verschiebe(zeichen, shift)

In [15]:
vigVerschiebe("H","S")

'Z'

Jetzt wenden wir diese Definitionen an, um einen Text mit einem Schlüsselwort per Vigenère-Chiffre zu verschlüsseln:

In [16]:
def vigenere(text, key):
    text = normiereText(text)
    key = normiereText(key)
    index = 0
    key_laenge = len(key)
    chiffre = ""
    for buchstabe in text:
        chiffre += vigVerschiebe(buchstabe, key[index])
        index += 1
        index %= key_laenge
    return chiffre

In [17]:
vigenere("HalloWelt","SCH")

'ZCSDQDWNA'

Um eine mit Vigenère verschlüsselte Botschaft wieder zu entschlüsseln, kann man - wie beim Caesar-Ciffre - die inverse Verschiebung nutzen:

In [18]:
def reVigVerschiebe(zeichen, schluesselzeichen):
    shift = indexVonZeichen(schluesselzeichen)  
    return verschiebe(zeichen, 26 - shift)

In [19]:
reVigVerschiebe("Z","S")

'H'

Damit kann jetzt die Entschlüsselung erfolgen:

In [20]:
def reVigenere(text, key):
    text = normiereText(text)
    key = normiereText(key)
    index = 0
    key_laenge = len(key)
    chiffre = ""
    for buchstabe in text:
        chiffre += reVigVerschiebe(buchstabe, key[index])
        index += 1
        index %= key_laenge
    return chiffre

Testen wir einmal, ob die Entschlüsselung den Klartext wiederherstellt:

In [21]:
botschaft = "LEISERIESELTDERSCHNEESTILLUNDSTARRRUHTDERSEE"
schluessel = "HUND"
geheim = vigenere(botschaft, schluessel)

klar = reVigenere(geheim, schluessel)
print(klar)

LEISERIESELTDERSCHNEESTILLUNDSTARRRUHTDERSEE
